# Как научить компьютер читать? 

В этой тетрадке мы обучим свой собственный word2vec. Делать мы это будем на каком-нибудь не очень большом тексте, который вам предстоит выбрать самому. На выбор есть [несколько сказок](https://github.com/nevmenandr/word2vec-russian-novels/tree/master/vector-school) и других [литературных штук](https://github.com/nevmenandr/word2vec-russian-novels/tree/master/books_before) из школьной программы. 

In [1]:
# Ссылка на выбранное вами произведение
# Я взял преступление и наказание 
url = 'https://raw.githubusercontent.com/nevmenandr/word2vec-russian-novels/master/books_before/CrimeAndPunishment.txt'


Спарсим текст из файлика.

In [2]:
import requests

resp = requests.get(url)
text = resp.text 

# Последние 500 символов. Аккуратно! Спойлеры!
print(text[-500:])

смотреть на эти семь лет, как на семь
дней. Он даже и не знал того, что новая жизнь не даром же ему достается, что ее надо еще дорого купить, заплатить за нее великим, будущим подвигом...
Но тут уж начинается новая история, история постепенного обновления человека, история постепенного перерождения его, постепенного перехода из одного мира в другой, знакомства с новою, доселе совершенно неведомою действительностью. Это могло бы составить тему нового рассказа, - но теперешний рассказ наш окончен.


## 1. Предобработка

Теперь нам надо его немного предобработать.  Пусть все слова пишутся с маленькой буквы. 

In [3]:
text = text.lower()

Разобьём весь текст на предложения. 

In [4]:
import re 
# выкидываем лишние символы! 
text = re.sub('\n|\t|\r', ' ', text)

In [5]:
from nltk.tokenize import sent_tokenize

sents = sent_tokenize(text)
len(sents)

13702

In [6]:
sents[220]

'действительно, на его платье и даже в волосах кое-где виднелись прилипшие былинки сена.'

Разобьём каждое предложение на отдельные слова.

In [7]:
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer('\w+')
tokenizer.tokenize(sents[220])

['действительно',
 'на',
 'его',
 'платье',
 'и',
 'даже',
 'в',
 'волосах',
 'кое',
 'где',
 'виднелись',
 'прилипшие',
 'былинки',
 'сена']

In [8]:
# разбейте все предложения на токены 
sents_tokenize  =  [tokenizer.tokenize(item) for item in sents]

In [9]:
# Flatten без numpy :) 
words = [item for sent in  sents_tokenize for item in sent]

In [10]:
len(words) # всего слов

173403

In [11]:
len(set(words)) # уникальных слов

24925

Можно выбросить все стоп-слова. 

In [12]:
from nltk.corpus import stopwords

stopwords_ru = stopwords.words('russian') 
stopwords_ru[:10]

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со']

In [13]:
len(stopwords_ru)

151

In [14]:
# избавьтесь от стоп-слов 
sents_tokenize = [[item for item in sent if item not in stopwords_ru]
                       for sent in sents_tokenize ]

Слов в корпусе не очень много. Давайте лемматизируем их.  В этом нам поможет библиотека **pymorphy2.**

**pymorphy2** — это полноценный морфологический анализатор, целиком написанный на Python. Он также умеет ставить слова в нужную форму (спрягать и склонять). [Документация по pymorphy2.](https://pymorphy2.readthedocs.io/en/latest/)

In [15]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

text = "Филипп пошёл в Авеньон и пленил пап!"
tokens = tokenizer.tokenize(text)

" ".join(morph.normal_forms(token)[0] for token in tokens)

'филипп пойти в авеньон и пленить папа'

In [16]:
p = morph.parse('стали')
p

[Parse(word='стали', tag=OpencorporaTag('VERB,perf,intr plur,past,indc'), normal_form='стать', score=0.984662, methods_stack=((<DictionaryAnalyzer>, 'стали', 904, 4),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn sing,gent'), normal_form='сталь', score=0.003067, methods_stack=((<DictionaryAnalyzer>, 'стали', 13, 1),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn sing,datv'), normal_form='сталь', score=0.003067, methods_stack=((<DictionaryAnalyzer>, 'стали', 13, 2),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn sing,loct'), normal_form='сталь', score=0.003067, methods_stack=((<DictionaryAnalyzer>, 'стали', 13, 5),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn plur,nomn'), normal_form='сталь', score=0.003067, methods_stack=((<DictionaryAnalyzer>, 'стали', 13, 6),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn plur,accs'), normal_form='сталь', score=0.003067, methods_stack=((<DictionaryAnalyzer>, 'стали', 13, 9),))]

In [17]:
morph.normal_forms('стали')

['стать', 'сталь']

Обработаем все слова из датасета. 

In [18]:
# лемматизируйте все слова из датасета
sents_tokenize = [[morph.normal_forms(item)[0] for item in sent] 
                    for sent in sents_tokenize]

In [19]:
# Flatten без numpy :) 
words = [item for sent in  sents_tokenize for item in sent]

In [20]:
len(words) # всего слов

93069

In [21]:
len(set(words)) # уникальных слов

11153

Хватит обработок! Мы тут не анализом текстов занимаемся, а нейросетками. Если хочешь больше предобработки, [хороший мануал](https://nbviewer.jupyter.org/github/FUlyankin/hse_texts_do/blob/master/sem_1/texts_sem1.ipynb) об этом.  Давайте построим словарик с частотностями и перейдём к моделированию. 

In [22]:
from collections import Counter

word_dict = Counter(words)
word_dict.most_common()[:20]

[('это', 1481),
 ('весь', 1405),
 ('один', 659),
 ('раскольник', 567),
 ('свой', 549),
 ('сказать', 544),
 ('человек', 501),
 ('говорить', 498),
 ('пётр', 492),
 ('знать', 478),
 ('стать', 441),
 ('мочь', 441),
 ('который', 430),
 ('самый', 430),
 ('такой', 395),
 ('очень', 387),
 ('какой', 379),
 ('соня', 379),
 ('рука', 369),
 ('дело', 363)]

In [23]:
words = word_dict.most_common()
len([item for item in words if item[1] >= 3])  # совсем мало :) 

4793

## 2. Моделирование

__Основные параметры:__

* данные должны быть итерируемым объектом 
* `size` — размер вектора, 
* `window` — размер окна наблюдения,
* `min_count` — мин. частотность слова в корпусе,
* `sg` — используемый алгоритм обучения (0 — CBOW, 1 — Skip-gram),
* `sample` — порог для downsampling'a высокочастотных слов,
* `workers` — количество потоков,
* `alpha` — learning rate,
* `iter` — количество итераций,
* `max_vocab_size` — позволяет выставить ограничение по памяти при создании словаря (т.е. если ограничение привышается, то низкочастотные слова будут выбрасываться). Для сравнения: 10 млн слов = 1Гб RAM.

In [24]:
#%%time 
from gensim.models.word2vec import Word2Vec

# size - размерность векторов, которые мы хотим обучить
# window - ширина окна контекста
# min_count - если слово встречается реже, для него не учим модель
model = Word2Vec(size=100, window=2, min_count=3, workers=-1)

# строительство словаря, чтобы обучение шло быстрее
model.build_vocab(sents_tokenize)

# обучение модели 
# первый аргумент - наша выборка, генератор будет вкидывать в модель наши тексты, пока они не кончатся
# второй аргумент - число примеров в выборке 
# третий аргумент - количество эпох обучения: сколько раз модель пройдётся по всему корпусу текстов
model.train(sents_tokenize, total_examples=model.corpus_count, epochs=100)

# !NB в ситуации, когда у нас огромный корпус, 100 эпох это слишком много! 

C:\Users\vvsol\AppData\Roaming\Python\Python37\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


(0, 0)

In [25]:
model.corpus_count # число примеров в обучающей выборке

13702

Смотрим, сколько в модели слов.

In [26]:
len(model.wv.vocab)

4793

In [27]:
'старуха' in model.wv.vocab

True

## 3. Свойства модели

In [28]:
# вектор слова
model.wv['старуха'][:10]

array([ 0.00269715, -0.00393186, -0.00253049, -0.00146901,  0.00316156,
        0.00128213, -0.00427795,  0.00365371,  0.00264595, -0.00187783],
      dtype=float32)

In [29]:
# размерность вектора
model.wv['старуха'].shape

(100,)

In [30]:
# похожести слов 
model.wv.similarity('тварь', 'право')

C:\Users\vvsol\AppData\Roaming\Python\Python37\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.064902365

In [31]:
# самые похожие
model.wv.most_similar('топор')

[('воровка', 0.4112016260623932),
 ('задать', 0.342094361782074),
 ('тревожиться', 0.3209070563316345),
 ('капернаумов', 0.3154948353767395),
 ('обидеть', 0.3093034327030182),
 ('высунуть', 0.2967184782028198),
 ('закутаться', 0.2924662232398987),
 ('пятьдесят', 0.2922254800796509),
 ('прочее', 0.2852746546268463),
 ('словечко', 0.28361421823501587)]

In [32]:
# арифметика
model.wv.most_similar(positive=['раскольников','соня'], 
                       negative=['тварь'])[:10]

[('напротив', 0.34410300850868225),
 ('кладбище', 0.327750563621521),
 ('состряпать', 0.3218076229095459),
 ('верить', 0.31428980827331543),
 ('наивный', 0.2964990735054016),
 ('худо', 0.2891800105571747),
 ('вид', 0.2863950729370117),
 ('поставить', 0.28526586294174194),
 ('зловещий', 0.28301507234573364),
 ('приглашать', 0.2803681492805481)]

## 4. Как дообучить модель? 

Ради чистоты эксперимента сохраним текущую модель и заново подгрузим её. 

In [33]:
model_path = "./our_w2v.model"
model.save(model_path)

In [34]:
our_model = Word2Vec.load(model_path)

Подгрузим другое произведение и сделаем для него предобработку. 

In [35]:
url = 'https://raw.githubusercontent.com/nevmenandr/word2vec-russian-novels/master/vector-school/SkazkaOCareSaltane.txt'

resp = requests.get(url)
text2 = resp.text 

# Последние 500 символов. Аккуратно! Спойлеры!
print(text2[-500:])

 узнает...
В нем взыграло ретивое!
"Что я вижу? что такое?
Как!" - и дух в нем занялся...
Царь слезами залился,
Обнимает он царицу,
И сынка, и молодицу,



И садятся все за стол;
И веселый пир пошел.
А ткачиха с поварихой,
С сватьей бабой Бабарихой
Разбежались по углам;
Их нашли насилу там.
Тут во всем они признались,
Повинились, разрыдались;
Царь для радости такой
Отпустил всех трех домой.
День прошел - царя Салтана
Уложили спать вполпьяна.
Я там был; мед, пиво пил -
И усы лишь обмочил.

1831




Предобработка.

In [36]:
# А теперь ваш код предобработки


In [37]:
sents_tokenize2[10]

NameError: name 'sents_tokenize2' is not defined

In [ ]:
len(sents_tokenize2)

Дополняем модель.

In [ ]:
# И теперь ваш код обучения

In [ ]:
'ядро' in model.wv.vocab

In [ ]:
'ядро' in our_model.wv.vocab

In [ ]:
our_model.wv.most_similar('ядро')

Пример со старым словом.

In [ ]:
our_model.wv.most_similar('сын')

In [ ]:
model.wv.most_similar('сын')